# Algo Final Program

## Imports

In [1]:
import math

## Fibonaci Heap Implementation

In [18]:
class FibonacciNode:
    def __init__(self, key):
        self.key = key
        self.parent = None
        self.child = None
        self.left = self
        self.right = self
        self.degree = 0
        self.mark = False

        
class FibonacciHeap:
    def __init__(self):
        self.root_list = None
        self.min_node = None
        self.total_nodes = 0

    '''
    returns the min value
    '''
    def get_min(self):
        return self.min_node.key

    '''
    removes and returns the min value
    '''
    def extract_min(self):
        min_node = self.min_node
        if min_node is not None:
            
            # If the minimum node has children, merge them into the root list
            if min_node.child is not None:
                children = [x for x in self._iterate(min_node.child)]
                for i in range(0, len(children)):
                    self._add_to_root_list(children[i])
                    children[i].parent = None
                    
            # remove from root list
            if min_node == self.root_list:
                # node is head of root list
                self.root_list = min_node.right
        
            # Remove the node from the doubly linked list
            min_node.left.right = min_node.right
            min_node.right.left = min_node.left
            
            # Update the min_node and consolidate the heap if needed
            if min_node == min_node.right:
                self.min_node = self.root_list = None
            else:
                self.min_node = min_node.right
                self._consolidate()
            self.total_nodes -= 1
        return min_node.key

    '''
    adds the param value to the root list
    returns the FibonacciNode created
    '''
    def insert(self, key):
        new_node = FibonacciNode(key)
        self._add_to_root_list(new_node)
        
        # Update the min_node if necessary
        if self.min_node is None or new_node.key < self.min_node.key:
            self.min_node = new_node
            
        self.total_nodes += 1
        return new_node

    '''
    given a FibonacciNode you can alter the value(key)
    '''
    def decrease_key(self, node, new_key):
        if new_key > node.key:
            return None
        
        node.key = new_key
        
        # Check if the node.key is less than its parent.key
        parent = node.parent
        if parent is not None and node.key < parent.key:
            self._cut(node, parent)
            self._cascading_cut(parent)
            
        # Update the min_node if necessary
        if node.key < self.min_node.key:
            self.min_node = node

    '''
    given 2 FibonacciHeap
    returns 1 combined heap
    '''
    def merge(self, heap_to_merge):
        this_heap = FibonacciHeap()
        
        # Merge root lists and update min node
        this_heap.root_list = self.root_list
        this_heap.min_node = self.min_node
        last = heap_to_merge.root_list.left
        heap_to_merge.root_list.left = this_heap.root_list.left
        this_heap.root_list.left.right = heap_to_merge.root_list
        this_heap.root_list.left = last
        this_heap.root_list.left.right = this_heap.root_list
        if heap_to_merge.min_node.key < this_heap.min_node.key:
            this_heap.min_node = heap_to_merge.min_node
        
        # Update total number of nodes
        this_heap.total_nodes = self.total_nodes + heap_to_merge.total_nodes
        return this_heap

    def _cascading_cut(self, node):
        parent = node.parent
        if parent is not None:
            if node.mark is False:
                node.mark = True
            else:
                # If the node is already marked, cut it from its parent and continue cascading
                self._cut(node, parent)
                self._cascading_cut(parent)

    def _consolidate(self):
        # Initialize degree_buckets to store nodes of each degree
        degree_buckets = [None] * int(math.log(self.total_nodes) * 2)
        
        # Collect all nodes of the Fibonacci heap
        all_nodes = [w for w in self._iterate(self.root_list)]
        
        # Iterate over each node in the Fibonacci heap
        for index_of_current in range(0, len(all_nodes)):
            current_node = all_nodes[index_of_current]
            current_degree = current_node.degree
            
            # Merge trees of equal degree
            while degree_buckets[current_degree] != None:
                other_node = degree_buckets[current_degree]
                
                # Swap nodes to maintain min-heap property
                if current_node.key > other_node.key:
                    temp = current_node
                    current_node, other_node = other_node, temp
                    
                self._heap_link(other_node, current_node)
                
                # Empty the degree bucket and move to the next degree
                degree_buckets[current_degree] = None
                current_degree += 1
                
            degree_buckets[current_degree] = current_node
            
        # Find the new minimum node in the root list
        for i in range(0, len(degree_buckets)):
            if degree_buckets[i] is not None:
                if degree_buckets[i].key < self.min_node.key:
                    self.min_node = degree_buckets[i]
                    
    def _cut(self, child, parent):
        # remove from child list
        if parent.child == parent.child.right:
            # parent has one chile
            parent.child = None
        elif parent.child == node:
            # node is first chile of parent
            parent.child = node.right
            node.right.parent = parent
            
        # Remove the node from the child list
        node.left.right = node.right
        node.right.left = node.left
        
        parent.degree -= 1
        self._add_to_root_list(child)
        child.parent = None
        child.mark = False
 
    def _iterate(self, begining):
        node = begining
        loop_completed = False
        while True:
            if node == begining:
                loop_completed = True
            yield node
            node = node.right
            if node == begining and loop_completed is True:
                break

    def _add_to_root_list(self, node):
        if self.root_list is None:
            # If root list is empty, set node as the root list
            self.root_list = node
        else:
            # Insert node into the root list
            node.right = self.root_list.right
            node.left = self.root_list
            self.root_list.right.left = node
            self.root_list.right = node
            
    def _heap_link(self, node1, node2):
        # Remove node1 from the root list
        if node1 == self.root_list:
            # node is head of root list
            self.root_list = node1.right
        
        # Remove the node from the doubly linked list
        node1.left.right = node1.right
        node1.right.left = node1.left
        
        # Reset node1's left and right references
        node1.left = node1
        node1.right = node1
        
        # Merge node1 into node2's child list
        parent = node2
        node = node1
        
        if node2.child is None:
            # If parent has no child, set node as the child
            node2.child = node1
        else:
            # Insert node into child list
            node1.right = node2.child.right
            node1.left = node2.child
            node2.child.right.left = node1
            node2.child.right = node1   
        
        node2.degree += 1
        
        # Update node1's parent and mark
        node1.parent = node2
        node1.mark = False


## Fibonacci Heap Validation

In [19]:
heap = FibonacciHeap()
t2 = heap.get_min()
t3 = heap.extract_min()
message = "pass" if t2 == t3 == None else "fail"
print("empty heap test:        " + message)

heap = FibonacciHeap()
heap.insert(1)
t2 = heap.get_min().key
t3 = heap.extract_min().key
message = "pass" if t2 == t3 == 1 else "fail"
print("1 value heap test:      " + message)

heap = FibonacciHeap()
heap.insert(5)
heap.insert(1)
t2 = heap.get_min().key
t3 = heap.extract_min().key
message = "pass" if t2 == t3 == 1 else "fail"
print("2 diff value heap test: " + message)

heap = FibonacciHeap()
heap.insert(1)
heap.insert(1)
t2 = heap.get_min().key
t3 = heap.extract_min().key
message = "pass" if t2 == t3 == 1 else "fail"
print("2 same heap test:       " + message)

heap = FibonacciHeap()
for n in range(1,10):
    heap.insert(n)

passed = True
for n in range(1,10):
    m = heap.extract_min().key
    if n != m:
        passed = False
        print("extract_min test:       Fail")
        break

if passed:
    print("extract_min test:       pass")
    
heap = FibonacciHeap()
node = heap.insert(5)
heap.insert(1)
heap.decrease_key(node, 0)

message = "pass" if heap.get_min().key == 0 else "fail"
print("decrease key heap test: " + message)

empty heap test:        pass
1 value heap test:      pass
2 diff value heap test: pass
2 same heap test:       pass
extract_min test:       pass
decrease key heap test: pass


## Fibonacci Heap Benchmarking

Operation | Amortized Running Time
----------|-----------------------
insert | $O(1)$
get_in | $O(1)$
extract_min | $O(log n)$
decrease_key | $O(1)$
merge | $O(1)$

